<a href="https://colab.research.google.com/github/kjapan87/ChatbotImplement_V1/blob/main/chatbot_with_training_module_12_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
#import tensorflow_addons as tfa
import re 
import time

In [ ]:
!pip install tensorflow==1.0.0

In [ ]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [ ]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [ ]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

In [ ]:
questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])


In [ ]:
def cleanText(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]"," ",text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"we'd", "we would", text)    
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [ ]:
clean_questions = []
for question in questions:
    clean_questions.append(cleanText(question))

In [ ]:
clean_answers = []
for answer in answers:
    clean_answers.append(cleanText(answer))

In [ ]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
          word2count[word] = 1
        else:
          word2count[word] += 1

In [ ]:
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
          word2count[word] = 1
        else:
          word2count[word] += 1 

In [ ]:
threshold_ques = 20
word_number = 0

In [ ]:
question2int_dict = {}
for word,count in word2count.items():
    if count >= threshold_ques:
        question2int_dict[word] = word_number
        word_number += 1

In [ ]:
threshold_ans = 20

In [ ]:
answers2int_dict = {}
for word,count in word2count.items():
    if count >= threshold_ans:
        answers2int_dict[word] = word_number
        word_number += 1    

In [ ]:
tokens = ["<PAD>","<EOS>","<OUT>","<SOS>"]

In [ ]:
for token in tokens:
    question2int_dict[token] = len(question2int_dict)+1

In [ ]:
for token in tokens:
    answers2int_dict[token] = len(answers2int_dict)+1

In [ ]:
answers2int_wordDict = {word_i:word for word,word_i in answers2int_dict.items()}

In [ ]:
for i in range(len(clean_answers)):
    clean_answers[i] += " <EOS>"

In [ ]:
out_questions = []

In [ ]:
for question in clean_questions:
  integ=[] #using full-sentences & using the created dictionarie
  for word in question.split():
    if word not in question2int_dict:
      integ.append(question2int_dict['<OUT>'])
    else:
      integ.append(question2int_dict[word])
  out_questions.append(integ)

In [ ]:
out_answers = []

In [ ]:
for answer in clean_answers:
  integ=[] #using full-sentences & using the created dictionaries
  for word in answer.split():
    if word not in answers2int_dict:
      integ.append(answers2int_dict['<OUT>'])
    else:
      integ.append(answers2int_dict[word])
  out_answers.append(integ)

In [ ]:
sorted_out_question = []
sorted_out_answers = []

In [ ]:
for length in range(1, 10+1):
    for i in enumerate(out_questions):
        if len(i[1]) == length:
            sorted_out_question.append(out_questions[i[0]])
            sorted_out_answers.append(out_answers[i[0]])

In [ ]:
def model_input():
    inputs = tf.placeholder(tf.int32,[None,None],name = "input")
    target = tf.placeholder(tf.int32,[None,None],name = "output")
    LR = tf.placeholder(tf.float32,name = "learning_rate")
    keep_prob = tf.placeholder(tf.float32,name = "dropout")
    return inputs,target,LR,keep_prob

In [ ]:
def preprocess (target,word2int,batch_size):
    left_side = tf.fill([batch_size,1], word2int["<SOS>"], name=None)
    right_side = tf.strided_slice(input = target, 
                                  begin = [0,0], 
                                  end = [batch_size,-1], 
                                  strides=[1,1], 
                                  begin_mask=0, 
                                  end_mask=0, 
                                  ellipsis_mask=0,
                                  new_axis_mask=0, 
                                  shrink_axis_mask=0, 
                                  var=None, name=None)
    pre_process_targets =tf.concat(values = [left_side,right_side], axis = 1)
    return pre_process_targets

In [ ]:
def encoder (rnn_input,
             rnn_size,
             num_layers,
             keep_prob,
             sequence_length):
    create_LSTM = tf.contrib.rnn.LSTMCell(rnn_size)
    LSTM_dropout = tf.contrib.rnn.DropoutWrapper(create_LSTM,input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([LSTM_dropout]*num_layers)
    _,encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw= encoder_cell, 
                                                      cell_bw= encoder_cell,
                                                      sequence_length = sequence_length,
                                                      inputs = rnn_input,
                                                      dtype = tf.float32)
    return encoder_state

In [ ]:
def decoder_training_data (encoder_state, 
                           decoder_cell, 
                           decoder_embedding_input, 
                           sequence_length, 
                           decoding_scope, 
                           output_function, 
                           keep_prob , 
                           batch_size):
    
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size], 
                                dtype=tf.dtypes.float32, 
                                name=None) 
    
    attention_keys, attention_values, attention_score_function, attention_construct_function \
        = tf.contrib.seq2seq.prepare_attention(attention_states,
                                               attention_option = "Bahdanau",
                                               num_units = decoder_cell.output_size,
                                               reuse=False)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], 
                                                                        attention_keys, 
                                                                        attention_values, 
                                                                        attention_score_function, 
                                                                        attention_construct_function, 
                                                                        name="AttentionDecoderTraining") 
    
    decoder_output,_,_ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, 
                                                                training_decoder_function, 
                                                                decoder_embedding_input, 
                                                                sequence_length, 
                                                                scope = decoding_scope, 
                                                                name="DecoderOutputforTraining")
    decoder_dropout = tf.nn.dropout(decoder_output,
                                    keep_prob,
                                    name="DecoderDropoutController")
    
    return output_function(decoder_dropout)

In [ ]:
def decoder_testing_data (encoder_state, 
                          decoder_cell, 
                          decoder_embedding_matrix, 
                          sos_id, 
                          eos_id, 
                          maximum_length, 
                          num_words, 
                          decoding_scope, 
                          output_function, 
                          keep_prob , 
                          batch_size):
    
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size], 
                                dtype=tf.dtypes.float32, 
                                name=None) 
    
    attention_keys, attention_values, attention_score_function, attention_construct_function \
        = tf.contrib.seq2seq.prepare_attention(attention_states, 
                                               attention_option = "bahdanau", 
                                               num_units = decoder_cell.output_size, 
                                               reuse=False) 
        
    testing_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function, 
                                                                           encoder_state[0], 
                                                                           attention_keys, 
                                                                           attention_values, 
                                                                           attention_score_function, 
                                                                           attention_construct_function, 
                                                                           decoder_embedding_matrix, 
                                                                           sos_id, 
                                                                           eos_id, 
                                                                           maximum_length, 
                                                                           num_words, 
                                                                           dtype=tf.int32, 
                                                                           name=None) 
      
    test_predictions,_,_ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, 
                                                                  testing_decoder_function, 
                                                                  scope = decoding_scope, 
                                                                  name="DecoderOutputforTesting")
    return test_predictions

In [ ]:
def decoder (decoder_embedding_input,
             decoder_embedding_matrix, 
             encoder_state, 
             num_words, 
             rnn_size, 
             num_layers, 
             word2int, 
             keep_prob, 
             sequence_length, 
             batch_size):
  with tf.variable_scope("decoding") as decoding_scope:
    create_LSTM = tf.contrib.rnn.LSTMCell(rnn_size)
        
    LSTM_dropout = tf.contrib.rnn.DropoutWrapper(create_LSTM,input_keep_prob = keep_prob)
        
    decoder_cell = tf.contrib.rnn.MultiRNNCell([LSTM_dropout]*num_layers)
    
    weights = tf.truncated_normal_initializer(mean=0.0,
                                                  stddev=1.0,
                                                  seed=None,
                                                  dtype=tf.dtypes.float32)
    bias = tf.zeros_initializer()
        
    output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      activation_fn=tf.nn.relu,
                                                                      normalizer_fn=None,
                                                                      normalizer_params=None,
                                                                      weights_initializer= weights,
                                                                                      weights_regularizer=None,
                                                                                      biases_initializer=bias,
                                                                                      biases_regularizer=None,
                                                                                      reuse=None,
                                                                                      variables_collections=None,
                                                                                      outputs_collections=None,
                                                                                      trainable=True,
                                                                                      scope=decoding_scope)
    training_predictions = decoder_training_data (encoder_state, 
                                                      decoder_cell, 
                                                      decoder_embedding_input, 
                                                      sequence_length, 
                                                      decoding_scope, 
                                                      output_function,
                                                      keep_prob , 
                                                      batch_size)
    decoding_scope.reuse_variables()

    testing_predictions = decoder_testing_data (encoder_state, 
                                                    decoder_cell, 
                                                    decoder_embedding_matrix, 
                                                    word2int["<SOS>"], 
                                                    word2int["<EOS>"],   
                                                    num_words, 
                                                    sequence_length-1, 
                                                    decoding_scope, 
                                                    output_function, 
                                                    keep_prob , 
                                                    batch_size)
        
    return training_predictions,testing_predictions
  

In [ ]:
def seq2seq (inputs,
             target,
             keep_prob,
             sequence_length, 
             batch_size, 
             questions_number_words,
             answers_number_words,
             encoder_embedding_size,
             decoder_embedding_size,
             rnn_size,
             num_layers,
             question2int_dict):
  encoder_embedded_input=tf.contrib.layers.embed_sequence(inputs,
                                                          answers_number_words+1,
                                                          encoder_embedding_size,
                                                          initializer=tf.random_uniform_initializer(minval=0, maxval=1,seed=None)
                                                          )
  encoder_state = encoder(encoder_embedded_input, 
                            rnn_size,
                            num_layers,
                            keep_prob,
                            sequence_length)
  pre_process_targets = preprocess(target, question2int_dict, batch_size)
  
  decoder_embedding_matrix= tf.Variable(tf.random.uniform([questions_number_words+1,decoder_embedding_size],
                                                          minval=0,
                                                          maxval=1,
                                                          dtype=tf.dtypes.float32,
                                                          seed=None,
                                                          name=None))
  
  decoder_embedding_input = tf.nn.embedding_lookup(decoder_embedding_matrix,
                                                   pre_process_targets,
                                                   max_norm=None,name=None)
  
  training_predictions,testing_predictions \
        = decoder (decoder_embedding_input,
                   decoder_embedding_matrix, 
                   encoder_state, 
                   questions_number_words, 
                   rnn_size, 
                   num_layers, 
                   question2int_dict, 
                   keep_prob, 
                   sequence_length, 
                   batch_size)
    
  return training_predictions,testing_predictions

In [ ]:
epochs = 10
batch_size = 64
rnn_size = 512 ##no. of neurons in 1 RNN cell
num_layers = 3
encoder_embedding_size = 512
decoder_embedding_size = 512
learning_rate = 0.01 
learning_rate_decay = 0.90 
min_learning_rate = 0.0001
keep_probability = 0.50

In [ ]:
#tf.compat.v1.disable_eager_execution()
tf.reset_default_graph()
session = tf.InteractiveSession()

In [ ]:
inputs,target,LR,keep_prob = model_input()

In [ ]:
sequence_length = tf.placeholder_with_default(25,[None,None,1,1], name='sequence_length')

In [ ]:
input_shape = tf.shape(inputs)
#out_type=tf.dtypes.int32, name=None
len_question = len(question2int_dict)
len_answer = len(answers2int_dict)

# ***ERROR NEXT CELL***







In [ ]:
training_predictions,testing_predictions\
    =seq2seq (tf.reverse(inputs,[-1]),
              target,
              keep_prob,
              batch_size, 
              sequence_length,
              len(answers2int_dict),
              len(question2int_dict),
              encoder_embedding_size,
              decoder_embedding_size,
              rnn_size,
              num_layers,
              question2int_dict)

ValueError: ignored

In [ ]:
with tf.name_scope("Optimization"):
  loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                         target,
                                         tf.ones([input_shape[0],sequence_length])
                                         )
  Optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
  Gradient = Optimizer.compute_gradients(loss_error)
  Clipped_Gradient = [(tf.clip_by_value(grad_tensor,-5.0,5.0),grad_variable) for grad_tensor,grad_variable in Gradient if grad_tensor is not None]
  Optimizer_Clipped_Gradient = Optimizer.apply_gradients(Clipped_Gradient)

In [ ]:
def apply_padding(batch_of_sequences, word2int):
  maximum_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
  return [sequence + [word2int["<PAD>"]]*(maximum_sequence_length-len(sequence)) for sequence in batch_of_sequences]

In [ ]:
def splits_into_batches(questions,answers,batch_size):
  for batch_index in range(0,len(questions)//batch_size):
    start_index = batch_index*batch_size
    question_in_batch = questions[start_index:start_index+batch_size]
    answers_in_batch  = answers[start_index:start_index+batch_size]
    padded_question_in_batch = np.array(apply_padding(question_in_batch, question2int_dict))
    padded_answer_in_batch = np.array(apply_padding(answers_in_batch,answers2int_dict))
  yield padded_question_in_batch,padded_answer_in_batch

In [ ]:
training_validation_split = int(len(sorted_out_question)*0.15)

training_questions = sorted_out_question[training_validation_split:]
training_answers = sorted_out_answers[training_validation_split:]
validation_questions = sorted_out_question[:training_validation_split]
validation_answers   = sorted_out_answers[:training_validation_split]

In [ ]:
batch_index_check_trainingLoss=100 
batch_index_check_validationLoss=(len(training_questions)//(batch_size//2))-1
list_validation_loss_error=[]
Total_training_loss_error=0
Total_validation_loss_error=0
early_stopping_check=0
early_stopping_stop=1000

In [ ]:
checkpoint = "./checkpoints.ckpt"

In [ ]:
session.run(tf.compat.v1.global_variables_initializer())

In [ ]:
for epoch in range(1,epochs+1):
    for batch_index,(padded_question_in_batch,padded_answer_in_batch) in enumerate(splits_into_batches(questions,answers,batch_size)):
        start_time=time.time()
        _,batch_training_loss_error=session.run([Optimizer_Clipped_Gradient,loss_error],{inputs:padded_question_in_batch, target:padded_answer_in_batch, LR:learning_rate,sequence_length:padded_answer_in_batch.shape[1], keep_prob:keep_probability})
        Total_training_loss_error+= batch_training_loss_error
        end_time=time.time()
        batch_time=end_time-start_time
        if batch_index%batch_index_check_trainingLoss==0:
            print("epoch:{:>3}/{},batch_size:{:>4}/{},training_loss_error:{:>6.3f},training_time_batches:{:d}seconds".format(epoch,epochs,batch_index,len(training_questions)//batch_size, Total_training_loss_error/batch_index_check_trainingLoss, int(batch_time*batch_index_check_trainingLoss)))
            Total_training_loss_error=0
            
        if batch_index%batch_index_check_validationLoss==0&batch_index>0:
            Total_validation_loss_error=0
            start_time=time.time()
            for batch_validation,(padded_question_in_batch,padded_answer_in_batch) in enumerate(splits_into_batches(validation_questions,validation_answers,batch_size)):
                batch_validation_loss_error=session.run(loss_error,{inputs:padded_question_in_batch, target:padded_answer_in_batch, LR:learning_rate,sequence_length:padded_answer_in_batch.shape[1], keep_prob:1})
                
                Total_validation_loss_error+= batch_validation_loss_error
                end_time=time.time()
                batch_time=end_time-start_time
                avg_validation_loss_error=Total_validation_loss_error/(len(validation_questions)/batch_size)
                
                print("validation_loss_error:{:>6.3f},validation_time_batches:{:d}seconds".format(avg_validation_loss_error, int(batch_time)))
                learning_rate *= learning_rate_decay
                
                if learning_rate<min_learning_rate:
                    learning_rate=min_learning_rate
                    list_validation_loss_error.append(avg_validation_loss_error)
                
                if avg_validation_loss_error<=min(list_validation_loss_error):
                    print("i speak better now")
                    early_stopping_check=0
                    saver=tf.compat.v1.train.Saver()
                    saver.save(session,checkpoint)
                else:
                    print("Sorry, I don't speak better, I need to practice more")
                    early_stopping_check+=1
                    if early_stopping_check==early_stopping_stop:
                        break
    if early_stopping_check==early_stopping_stop:
        print("My apologies, i can't speak better anymore, this is the best i can do")
        break
print("conversation over")